In [1]:
def is_colab():
    """Check if the code is running in Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Use the function to conditionally run magic commands
if is_colab():
    # Run Colab-specific magic commands
    print("Running in Colab, executing magic commands.")
    !rm -rf microproyecto3NLP/
    !git clone https://github.com/cjohana031/microproyecto3NLP
    !cp -R microproyecto3NLP/* .
    # Add any other Colab-specific setup
else:
    # Alternative setup for non-Colab environments
    print("Not running in Colab, nothing else is needed.")

Running in Colab, executing magic commands.
Cloning into 'microproyecto3NLP'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 34.20 MiB | 31.01 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
with open('data/test.txt', 'r') as f:
  print(f.read())

holii


In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [13]:
# Importación de librerías
import sentencepiece as spm
import io
import time
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,BertForQuestionAnswering
)

# Configurar semillas para facilitar la reproducibilidad de los resultados
seed = 99
torch.manual_seed(seed)
np.random.seed(seed)

In [9]:
#read dataset test

test_path = 'data/test-00000-of-00001.parquet'
test = pd.read_parquet(test_path)
test = test[test['article'].str.len() < 800]
test



,example_id,article,answer,question,options
45,high18377.txt,A new law helps people with disabilities. The ...,D,According to the passage we can see that_.,[it will be difficult for the normal persons t...
46,high18377.txt,A new law helps people with disabilities. The ...,C,Ramps can help people_.,"[with hearing problems, who have difficulty in..."
47,high18377.txt,A new law helps people with disabilities. The ...,D,"The word ""Braille"" means_.","[words known by foreigners, words known by com..."
48,high18377.txt,A new law helps people with disabilities. The ...,C,What do you think the disabled would feel abou...,"[They may think the new law reasonable., Some ..."
49,high18377.txt,A new law helps people with disabilities. The ...,B,The fact that the new law has been passed show...,[the government has paid no attention to the d...
...,...,...,...,...,...
4915,middle926.txt,"Most People don't like mice, but they love one...",C,"In the cartoon, Pluto is a _ dog.","[clever, smart, foolish, bad]"
4916,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,A,Who is the oldest of the four boys?,"[Dick., David., Dean., Denis.]"
4917,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,D,What do the four boys like doing every day?,"[Reading stories., Playing computer games., Wa..."
4918,middle4599.txt,Mrs. Black has four sons. Dick is six years ol...,C,What will they do on Mrs. Black's birthday?,"[Eat out., Play soccer., Have a party., Play c..."


In [10]:
#read dataset train

train_path = 'data/train-00000-of-00001.parquet'
train = pd.read_parquet(train_path)
train = train[train['article'].str.len() < 800]
train

,example_id,article,answer,question,options
244,high2778.txt,The air hostess was in a small kitchen at th...,C,The story happened _ .,"[in the evening, in the afternoon, in the morn..."
245,high2778.txt,The air hostess was in a small kitchen at th...,A,What were the four men doing when the little o...,"[They were busy working in the control room., ..."
246,high2778.txt,The air hostess was in a small kitchen at th...,B,Which of the following is TRUE according to th...,"[The air hostess was humorous ., The old lady..."
329,high15490.txt,"Moscow,Russia(Space news)-""The computer is a b...",D,Which of the following best gives the main ide...,"[5,000 dollars goes to a computer!, New invent..."
330,high15490.txt,"Moscow,Russia(Space news)-""The computer is a b...",C,What was it that Prozorov felt most bitter about?,"[That he didn't win the$5,000., That he hadn't..."
...,...,...,...,...,...
87829,middle4209.txt,How to get along with people you don't like? W...,D,This passage is most probably from _ .,"[a map, a cook book, a story book, a magazine]"
87846,middle4809.txt,Mrs Green lives in a small village. Her husban...,D,Green Sea is _ the small village.,"[in, next to, near, far from]"
87847,middle4809.txt,Mrs Green lives in a small village. Her husban...,B,That day Mrs Green goes to Green Sea _ .,"[by bike, by train, by air, by bus]"
87848,middle4809.txt,Mrs Green lives in a small village. Her husban...,D,Jack doesn't telephone his mother because _ .,"[he doesn't like her, he can see her every day..."


In [14]:
# Cargar modelo y tokenizador

model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Configurar tokenizador
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B.
401 Client Error. (Request ID: Root=1-681d02c0-3fe3cec90be551d863c4f012;3e997f02-26b7-4398-8278-f73e877bcdc1)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in.